<a href="https://colab.research.google.com/github/RossM/pix2pix-denoising-colab/blob/main/pix2pix_denoising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## GPU status

In [ ]:
import subprocess
simple_nvidia_smi_display = False#@param {type:"boolean"}
if simple_nvidia_smi_display:
    #!nvidia-smi
    nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
else:
    #!nvidia-smi -i 0 -e 0
    nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
    nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_ecc_note)

## Set up Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Download the data

In [ ]:
DATA_DIR = "/content/danbooru2021/images"

!mkdir -p $DATA_DIR
!rsync --progress --recursive --size-only --verbose rsync://176.9.41.242:873/danbooru2021/512px/000*/ $DATA_DIR/512px/


## Install dependencies

In [ ]:
!pip install dominate wandb torchvision einops realesrgan denoising-diffusion-pytorch

## Define our model

In [ ]:
import os
import torch
import torch.nn as nn
from torch.nn import init
from torch.optim import lr_scheduler
from denoising_diffusion_pytorch import Unet
from realesrgan.archs.discriminator_arch import UNetDiscriminatorSN

# Based on code from pytorch-pix2pix https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

def init_weights(net, init_type='normal', init_gain=0.02):
    """Initialize network weights.

    Parameters:
        net (network)   -- network to be initialized
        init_type (str) -- the name of an initialization method: normal | xavier | kaiming | orthogonal
        init_gain (float)    -- scaling factor for normal, xavier and orthogonal.

    We use 'normal' in the original pix2pix and CycleGAN paper. But xavier and kaiming might
    work better for some applications. Feel free to try yourself.
    """
    def init_func(m):  # define the initialization function
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            if init_type == 'normal':
                init.normal_(m.weight.data, 0.0, init_gain)
            elif init_type == 'xavier':
                init.xavier_normal_(m.weight.data, gain=init_gain)
            elif init_type == 'kaiming':
                init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
            elif init_type == 'orthogonal':
                init.orthogonal_(m.weight.data, gain=init_gain)
            else:
                raise NotImplementedError('initialization method [%s] is not implemented' % init_type)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
        elif classname.find('BatchNorm2d') != -1:  # BatchNorm Layer's weight is not a matrix; only normal distribution applies.
            init.normal_(m.weight.data, 1.0, init_gain)
            init.constant_(m.bias.data, 0.0)

    print('initialize network with %s' % init_type)
    net.apply(init_func)  # apply the initialization function <init_func>


def init_net(net, init_type='normal', init_gain=0.02, gpu_ids=[]):
    """Initialize a network: 1. register CPU/GPU device (with multi-GPU support); 2. initialize the network weights
    Parameters:
        net (network)      -- the network to be initialized
        init_type (str)    -- the name of an initialization method: normal | xavier | kaiming | orthogonal
        gain (float)       -- scaling factor for normal, xavier and orthogonal.
        gpu_ids (int list) -- which GPUs the network runs on: e.g., 0,1,2

    Return an initialized network.
    """
    if len(gpu_ids) > 0:
        assert(torch.cuda.is_available())
        net.to(gpu_ids[0])
        net = torch.nn.DataParallel(net, gpu_ids)  # multi-GPUs
    init_weights(net, init_type, init_gain=init_gain)
    return net

def get_scheduler(optimizer, opt):
    """Return a learning rate scheduler

    Parameters:
        optimizer          -- the optimizer of the network
        opt (option class) -- stores all the experiment flags; needs to be a subclass of BaseOptions．　
                              opt.lr_policy is the name of learning rate policy: linear | step | plateau | cosine

    For 'linear', we keep the same learning rate for the first <opt.n_epochs> epochs
    and linearly decay the rate to zero over the next <opt.n_epochs_decay> epochs.
    For other schedulers (step, plateau, and cosine), we use the default PyTorch schedulers.
    See https://pytorch.org/docs/stable/optim.html for more details.
    """
    def lambda_rule(epoch):
        lr_l = 1.0 - max(0, epoch + opt.epoch_count - opt.n_epochs) / float(opt.n_epochs_decay + 1)
        return lr_l
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_rule)
    return scheduler


##############################################################################
# Classes
##############################################################################
class GANLoss(nn.Module):
    """Define different GAN objectives.

    The GANLoss class abstracts away the need to create the target label tensor
    that has the same size as the input.
    """

    def __init__(self, gan_mode, target_real_label=1.0, target_fake_label=0.0):
        """ Initialize the GANLoss class.

        Parameters:
            gan_mode (str) - - the type of GAN objective. It currently supports vanilla, lsgan, and wgangp.
            target_real_label (bool) - - label for a real image
            target_fake_label (bool) - - label of a fake image

        Note: Do not use sigmoid as the last layer of Discriminator.
        LSGAN needs no sigmoid. vanilla GANs will handle it with BCEWithLogitsLoss.
        """
        super(GANLoss, self).__init__()
        self.register_buffer('real_label', torch.tensor(target_real_label))
        self.register_buffer('fake_label', torch.tensor(target_fake_label))
        self.gan_mode = gan_mode
        if gan_mode == 'lsgan':
            self.loss = nn.MSELoss()
        elif gan_mode == 'vanilla':
            self.loss = nn.BCEWithLogitsLoss()
        elif gan_mode in ['wgangp']:
            self.loss = None
        else:
            raise NotImplementedError('gan mode %s not implemented' % gan_mode)

    def get_target_tensor(self, prediction, target_is_real):
        """Create label tensors with the same size as the input.

        Parameters:
            prediction (tensor) - - tpyically the prediction from a discriminator
            target_is_real (bool) - - if the ground truth label is for real images or fake images

        Returns:
            A label tensor filled with ground truth label, and with the size of the input
        """

        if target_is_real:
            target_tensor = self.real_label
        else:
            target_tensor = self.fake_label
        return target_tensor.expand_as(prediction)

    def __call__(self, prediction, target_is_real):
        """Calculate loss given Discriminator's output and grount truth labels.

        Parameters:
            prediction (tensor) - - tpyically the prediction output from a discriminator
            target_is_real (bool) - - if the ground truth label is for real images or fake images

        Returns:
            the calculated loss.
        """
        if self.gan_mode in ['lsgan', 'vanilla']:
            target_tensor = self.get_target_tensor(prediction, target_is_real)
            loss = self.loss(prediction, target_tensor)
        elif self.gan_mode == 'wgangp':
            if target_is_real:
                loss = -prediction.mean()
            else:
                loss = prediction.mean()
        return loss

class Pix2PixModel:
    """ This class implements the pix2pix model, for learning a mapping from input images to output images given paired data.

    The model training requires '--dataset_mode aligned' dataset.
    By default, it uses a '--netG unet256' U-Net generator,
    a '--netD basic' discriminator (PatchGAN),
    and a '--gan_mode' vanilla GAN loss (the cross-entropy objective used in the orignal GAN paper).

    pix2pix paper: https://arxiv.org/pdf/1611.07004.pdf
    """
    def __init__(self, opt):
        """Initialize the pix2pix class.

        Parameters:
            opt (Option class)-- stores all the experiment flags; needs to be a subclass of BaseOptions
        """
        # Base model
        self.opt = opt
        self.gpu_ids = opt.gpu_ids
        self.isTrain = opt.isTrain
        self.device = torch.device('cuda:{}'.format(self.gpu_ids[0])) if self.gpu_ids else torch.device('cpu')  # get device name: CPU or GPU
        self.save_dir = os.path.join(opt.checkpoints_dir, opt.name)  # save all the checkpoints to save_dir
        torch.backends.cudnn.benchmark = True
        self.optimizers = []
        self.image_paths = []
        self.metric = 0  # used for learning rate policy 'plateau'

        # specify the training losses you want to print out. The training/test scripts will call <BaseModel.get_current_losses>
        self.loss_names = ['G_GAN', 'G_L1', 'D_real', 'D_fake']
        # specify the images you want to save/display. The training/test scripts will call <BaseModel.get_current_visuals>
        self.visual_names = ['real_A', 'fake_B', 'real_B']
        # specify the models you want to save to the disk. The training/test scripts will call <BaseModel.save_networks> and <BaseModel.load_networks>
        if self.isTrain:
            self.model_names = ['G', 'D']
        else:  # during test time, only load G
            self.model_names = ['G']
        # define networks (both generator and discriminator)
        self.netG = Unet(dim=opt.ngf, resnet_block_groups=8)
        #self.netG.bfloat16()
        self.netG = init_net(self.netG, opt.init_type, opt.init_gain, opt.gpu_ids)

        if self.isTrain:  # define a discriminator; conditional GANs need to take both input and output images; Therefore, #channels for D is input_nc + output_nc
            #self.netD = Unet(channels=opt.input_nc+opt.output_nc, dim=opt.ndf, resnet_block_groups=4, out_dim=1)
            #self.netD.bfloat16()
            self.netD = UNetDiscriminatorSN(num_in_ch=opt.input_nc+opt.output_nc, num_feat=opt.ndf)
            self.netD = init_net(self.netD, opt.init_type, opt.init_gain, opt.gpu_ids)

        if self.isTrain:
            # define loss functions
            self.criterionGAN = GANLoss(opt.gan_mode).to(self.device)
            self.criterionL1 = torch.nn.L1Loss()
            self.criterionMSE = torch.nn.MSELoss()
            # initialize optimizers; schedulers will be automatically created by function <BaseModel.setup>.
            self.optimizer_G = torch.optim.Adam(self.netG.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
            self.optimizer_D = torch.optim.Adam(self.netD.parameters(), lr=opt.lr, betas=(opt.beta1, 0.999))
            self.optimizers.append(self.optimizer_G)
            self.optimizers.append(self.optimizer_D)

    def forward(self, time = None):
        """Run forward pass; called by both functions <optimize_parameters> and <test>."""
        self.fake_B = self.netG(self.real_A, time)  # G(A)

    def backward_D(self, time = None):
        """Calculate GAN loss for the discriminator"""
        # Fake; stop backprop to the generator by detaching fake_B
        fake_AB = torch.cat((self.real_A, self.fake_B), 1)  # we use conditional GANs; we need to feed both input and output to the discriminator
        pred_fake = self.netD(fake_AB.detach())
        self.loss_D_fake = self.criterionGAN(pred_fake, False)
        # Real
        real_AB = torch.cat((self.real_A, self.real_B), 1)
        pred_real = self.netD(real_AB)
        self.loss_D_real = self.criterionGAN(pred_real, True)
        # combine loss and calculate gradients
        self.loss_D = (self.loss_D_fake + self.loss_D_real) * 0.5
        self.loss_D.backward()

    def backward_G(self, time = None):
        """Calculate GAN and L1 loss for the generator"""
        # First, G(A) should fake the discriminator
        fake_AB = torch.cat((self.real_A, self.fake_B), 1)
        pred_fake = self.netD(fake_AB)
        self.loss_G_GAN = self.criterionGAN(pred_fake, True)
        # Second, G(A) = B
        self.loss_G_L1 = self.criterionL1(self.fake_B, self.real_B) * self.opt.lambda_L1
        self.loss_G_MSE = self.criterionMSE(self.fake_B, self.real_B) * self.opt.lambda_MSE
        # combine loss and calculate gradients
        self.loss_G = self.loss_G_GAN + self.loss_G_L1 + self.loss_G_MSE
        self.loss_G.backward()

    def optimize_parameters(self, time=None):
        #time.bfloat16()
        self.forward(time)                   # compute fake images: G(A)
        # update D
        self.set_requires_grad(self.netD, True)  # enable backprop for D
        self.optimizer_D.zero_grad()     # set D's gradients to zero
        self.backward_D(time)                # calculate gradients for D
        self.optimizer_D.step()          # update D's weights
        # update G
        self.set_requires_grad(self.netD, False)  # D requires no gradients when optimizing G
        self.optimizer_G.zero_grad()        # set G's gradients to zero
        self.backward_G(time)                   # calculate graidents for G
        self.optimizer_G.step()             # udpate G's weights

    def setup(self, opt):
        """Load and print networks; create schedulers

        Parameters:
            opt (Option class) -- stores all the experiment flags; needs to be a subclass of BaseOptions
        """
        if self.isTrain:
            self.schedulers = [get_scheduler(optimizer, opt) for optimizer in self.optimizers]
        self.print_networks(opt.verbose)

    def eval(self):
        """Make models eval mode during test time"""
        for name in self.model_names:
            if isinstance(name, str):
                net = getattr(self, 'net' + name)
                net.eval()

    def test(self):
        """Forward function used in test time.

        This function wraps <forward> function in no_grad() so we don't save intermediate steps for backprop
        It also calls <compute_visuals> to produce additional visualization results
        """
        with torch.no_grad():
            self.forward()
            self.compute_visuals()

    def update_learning_rate(self):
        """Update learning rates for all the networks; called at the end of every epoch"""
        old_lr = self.optimizers[0].param_groups[0]['lr']
        for scheduler in self.schedulers:
            scheduler.step()

        lr = self.optimizers[0].param_groups[0]['lr']
        print('learning rate %.7f -> %.7f' % (old_lr, lr))

    def save_networks(self, epoch):
        """Save all the networks to the disk.

        Parameters:
            epoch (int) -- current epoch; used in the file name '%s_net_%s.pth' % (epoch, name)
        """
        for name in self.model_names:
            if isinstance(name, str):
                save_filename = '%s_net_%s.pth' % (epoch, name)
                save_path = os.path.join(self.save_dir, save_filename)
                net = getattr(self, 'net' + name)

                if len(self.gpu_ids) > 0 and torch.cuda.is_available():
                    torch.save(net.module.cpu().state_dict(), save_path)
                    net.cuda(self.gpu_ids[0])
                else:
                    torch.save(net.cpu().state_dict(), save_path)

    def load_networks(self, epoch):
        """Load all the networks from the disk.

        Parameters:
            epoch (int) -- current epoch; used in the file name '%s_net_%s.pth' % (epoch, name)
        """
        for name in self.model_names:
            if isinstance(name, str):
                load_filename = '%s_net_%s.pth' % (epoch, name)
                load_path = os.path.join(self.save_dir, load_filename)
                net = getattr(self, 'net' + name)
                if isinstance(net, torch.nn.DataParallel):
                    net = net.module
                print('loading the model from %s' % load_path)
                # if you are using PyTorch newer than 0.4 (e.g., built from
                # GitHub source), you can remove str() on self.device
                state_dict = torch.load(load_path, map_location=str(self.device))
                if hasattr(state_dict, '_metadata'):
                    del state_dict._metadata

                net.load_state_dict(state_dict)

    def print_networks(self, verbose):
        """Print the total number of parameters in the network and (if verbose) network architecture

        Parameters:
            verbose (bool) -- if verbose: print the network architecture
        """
        print('---------- Networks initialized -------------')
        for name in self.model_names:
            if isinstance(name, str):
                net = getattr(self, 'net' + name)
                num_params = 0
                for param in net.parameters():
                    num_params += param.numel()
                if verbose:
                    print(net)
                print('[Network %s] Total number of parameters : %.3f M' % (name, num_params / 1e6))
        print('-----------------------------------------------')

    def set_requires_grad(self, nets, requires_grad=False):
        """Set requies_grad=Fasle for all the networks to avoid unnecessary computations
        Parameters:
            nets (network list)   -- a list of networks
            requires_grad (bool)  -- whether the networks require gradients or not
        """
        if not isinstance(nets, list):
            nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad



# Model configuration

In [ ]:
class TrainOptions():
    pass

opt = TrainOptions()
opt.checkpoints_dir = '/content/drive/MyDrive/AI/pix2pix/checkpoints'
opt.name = "pix2pix-denoising-4"

# Model parameters. Changing these will invalidate checkpoint files
opt.input_nc = opt.output_nc = 3
opt.ngf = 96
opt.ndf = 128

# Training parameters
opt.init_type = "normal"
opt.init_gain = 0.02
opt.gan_mode = "lsgan"
opt.beta1 = 0.5
opt.lr = 0.0002
opt.epoch_count = 1
opt.n_epochs = 100
opt.n_epochs_decay = 100
opt.lambda_L1 = 0.0
opt.lambda_MSE = 100.0
opt.verbose = False       #@param {type: 'boolean'}
opt.gpu_ids = [0]
opt.noise_min = 0.1
opt.noise_max = 3.0
opt.noise_alpha = 2.0

# Input parameters. Changing these may degrade result quality for a few
# hundred batches until the model adjusts.
opt.noise_schedule_beta = 0.003     # Used for calculating the time passed to the generator
opt.use_random_times = False      # Estimates time based on post-noise image statistics
opt.clamp_input = False


# Train

In [ ]:
import math, random
import torch
import torchvision as tv
import einops
from torchvision.transforms.functional import to_pil_image
from torchvision.datasets import ImageFolder
from IPython import display

torch.cuda.set_device(0)
cuda = torch.device('cuda')

dataset = tv.datasets.ImageFolder(root = DATA_DIR, 
  transform = tv.transforms.Compose([
    #tv.transforms.Resize(256),                                                            
    tv.transforms.ToTensor()
  ]))
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=4)


opt.isTrain = True
opt.continue_train = False

RUN_DIR = f"{opt.checkpoints_dir}/{opt.name}"
!mkdir -p $RUN_DIR

model = Pix2PixModel(opt)
model.setup(opt)

start_epoch = 1
last_epoch = 100
# Resume
try:
  cp = open(f"{opt.checkpoints_dir}/{opt.name}/latest_epoch", "r")
  start_epoch = int(cp.read())
  model.load_networks(str(start_epoch))
  start_epoch += 1
  cp.close()
except:
  pass

#display.clear_output(wait=True)
#dataset[1000][0]

crop = tv.transforms.RandomCrop(256)

for epoch in range(start_epoch, last_epoch+1):
  model.update_learning_rate()
  for i, data in enumerate(data_loader):
    output_data = crop(data[0].cuda()) * 2 - 1
    stddev = torch.rand(size = (output_data.size()[0], ), device = cuda) ** opt.noise_alpha * (opt.noise_max - opt.noise_min) + opt.noise_min
    input_data = (output_data + torch.randn_like(output_data) * einops.rearrange(stddev, 'd -> d 1 1 1'))
    if opt.clamp_input:
      input_data = input_data.clamp(-1, 1)

    model.real_A = input_data
    model.real_B = output_data

    # Time estimation based on an exponentially decreasing cumulative beta schedule with constant beta
    if opt.use_random_times:
      time = torch.rand(size = (output_data.size()[0], )) * 1000
    else:
      time = torch.log(stddev / (1 + stddev)) / math.log(1 - opt.noise_schedule_beta)
    model.optimize_parameters(time)

    if i % 10 == 0:
      display.clear_output(wait=True)
      model.print_networks(verbose=opt.verbose)
      print(f"epoch {epoch}/{last_epoch} batch {i}/{len(data_loader)}")
      sample_index = 0
      print(f"Sample {sample_index}: stddev {stddev[sample_index]}, time {time[sample_index]}")
      display.display_png(to_pil_image(torch.cat((model.real_A[0],
                                                  model.real_B[0],
                                                  model.fake_B[0]), dim=2).clamp(-1, 1) * 0.5 + 0.5))
      print(f"Batch losses: G_GAN {model.loss_G_GAN}, G_L1 {model.loss_G_L1}, G_MSE {model.loss_G_MSE}, D_fake {model.loss_D_fake}, D_real {model.loss_D_real}")

  model.save_networks('latest')
  model.save_networks(epoch)
  cp = open(f"{opt.checkpoints_dir}/{opt.name}/latest_epoch", "w")
  cp.write(str(epoch))
  cp.close()


# Denoise

In [ ]:
import math, io, requests
import torch
import einops
from PIL import Image
from torchvision.transforms.functional import to_tensor, to_pil_image, resize
from IPython import display

image_url = "https://i.imgur.com/3NHuxu1.png" #@param {type: "string"}
image_size = [256, 256] #@param {type: "raw"}
injected_noise =  0.2#@param {type: "number"}
iterations = 1 #@param {type: "integer", min:1}

def fetch(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        r = requests.get(url_or_path)
        r.raise_for_status()
        fd = io.BytesIO()
        fd.write(r.content)
        fd.seek(0)
        return fd
    return open(url_or_path, 'rb')

torch.cuda.set_device(0)
cuda = torch.device('cuda')

opt.isTrain = False

model = Pix2PixModel(opt)
model.setup(opt)
model.load_networks('latest')

if image_url != "":
  pil_image = Image.open(fetch(image_url)).convert('RGB')

  print("Input image")
  display.display_png(pil_image)

  if image_size != None:
    pil_image = resize(pil_image, image_size)
  image = to_tensor(pil_image).to(model.device)
  image = einops.rearrange(image, '... -> 1 ...')
  image = image * 2 - 1
else:
  if image_size == None:
    image_size = [256, 256]
  image = torch.zeros(1, 3, image_size[0], image_size[1]).to(model.device)

image += torch.randn_like(image) * injected_noise
if opt.clamp_input:
  image = image.clamp(-1, 1)

model.set_requires_grad(model.netG, False)

if opt.use_random_times:
  time = torch.rand(size = (output_data.size()[0], )) * 1000
else:
  time = torch.log(injected_noise / (1 + injected_noise)) / math.log(1 - opt.noise_schedule_beta)

for step in range(0, iterations):
  t = step / iterations

  iter = model.netG(image, time)
  image = image + (iter - image) / (iterations - step)

  if step + 1 < iterations:
    print(f"Iteration {step+1}/{iterations}")
    display.display_png(to_pil_image(iter[0].clamp(-1, 1) * 0.5 + 0.5))

print("Final result")
display.display_png(to_pil_image(image[0].clamp(-1, 1) * 0.5 + 0.5))
